In [1]:
from selenium import webdriver
from time import sleep

In [43]:
browser = webdriver.Firefox()
browser.get('http://www.instagram.com/')

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("all_brand_models.csv", header = None, names=["brand","models"])

In [ ]:
brands = list(df.brand)

### Getting instagram accounts: number of posts, followers, following

In [6]:
from selenium.webdriver.common.keys import Keys
def search_for_brand(browser, brand):
    searchbar = browser.find_element_by_xpath('/html/body/span/section/nav/div[2]/div/div/div[2]/input')
    searchbar.send_keys(brand)
    sleep(0.5)
    searchbar.send_keys(Keys.RETURN)
    sleep(0.5)
    searchbar.send_keys(Keys.RETURN)
    sleep(0.5)
    try:
        searchbar.send_keys(Keys.RETURN)
    except:
        pass

In [31]:
browser.find_element_by_css_selector('h1[class="_rf3jb notranslate"]').text

'sony'

In [67]:
ig_name_css = 'h1[class="_rf3jb notranslate"]'
post_follower_following_css = 'span[class="_fd86t"]'
description_css = 'div[class="_tb97a"]'

In [51]:
browser.find_elements_by_css_selector(n_followers_css)[0].text
browser.find_elements_by_css_selector(n_followers_css)[1].text

'124'

In [7]:
def grab_stats(ig_name_css,post_follower_following_css, description_css):
    ig_name = browser.find_element_by_css_selector(ig_name_css).text
    n_posts = browser.find_elements_by_css_selector(post_follower_following_css)[0].text
    n_followers = browser.find_elements_by_css_selector(post_follower_following_css)[1].text
    n_following = browser.find_elements_by_css_selector(post_follower_following_css)[2].text
    description = browser.find_element_by_css_selector(description_css).text
    return ig_name, n_posts, n_followers, n_following, description

In [75]:
brand_igs = []
for brand in brands:
    browser.get("http://www.instagram.com/")
    sleep(1)
    search_for_brand(browser, brand)
    sleep(1)
    row = [brand]
    for attempts in range(3):
        sleep(1)
        try:
            stats = list(grab_stats(ig_name_css,post_follower_following_css, description_css))
        except:
            stats = ["-","-","-","-","-"]
            sleep(1)
    row += stats
    brand_igs.append(row)
    sleep(2)
    

In [71]:
grab_stats(ig_name_css,post_follower_following_css, description_css)

('plume_pt',
 '199',
 '8,453',
 '2,436',
 'Plume Loja de roupa disponível para entrega imediata ✨ worldwide shipping 📩 Não enviamos a cobrança')

In [76]:
brand_ig_df = pd.DataFrame(brand_igs)

In [78]:
brand_ig_df.to_csv("brand_ig_data.csv",index=False)

Manually filled in 10+ rows, some brands do not have instagram accounts

In [8]:
new_df = pd.read_csv("brand_ig_data.csv")

In [9]:
new_df.n_posts.apply(lambda x: 0 if x=="-" else int(x)).sum()

20993

In [190]:
element_to_hover_over= browser.find_element_by_css_selector('div[class="_mck9w _gvoze _tn0ps"]')

### Hovering over a post to get the number of likes, comments

This is a relatively easy way to get the comments as opposed to the more tedious method of visiting the post url and clicking many elements that may or may not exist.

In [2]:
from selenium.webdriver.common.action_chains import ActionChains

In [194]:
hover = ActionChains(browser).move_to_element(element_to_hover_over)
hover.perform()

In [198]:
browser.find_element_by_css_selector('div[class="_mli86"]').text.split("\n")

['0', 'likes', '3', 'comments']

In [200]:
new_df.brand_ig[0]

'samsungmobile'

In [12]:
insta_url = 'http://www.instagram.com/'

In [13]:
browser.get(insta_url +new_df.brand_ig[0])

In [290]:
elements_to_hover_over= browser.find_elements_by_css_selector('div[class="_mck9w _gvoze _tn0ps"]')

grab the likes, comments by hovering and then grab the link and caption

In [14]:
hover_css = 'div[class="_mli86"]'
def get_info(hover_css, element_to_hover_over):
    browser.execute_script("return arguments[0].scrollIntoView(true);", element_to_hover_over)
    hover = ActionChains(browser).move_to_element(element_to_hover_over)
    hover.perform()
    sleep(1)
    info = browser.find_element_by_css_selector(hover_css).text.split("\n")
    
    if info[1] == "likes":
        media_type = "image"
        views = "na"
        likes = info[0]
        
    else:
        media_type = "video"
        views = info[0]
        likes = "na"

    if len(info) == 2:
        comments = None
    else:
        comments = info[2]
    link = element_to_hover_over.find_element_by_tag_name("a").get_attribute("href")
    caption = element_to_hover_over.find_element_by_tag_name("img").get_attribute("alt")
    return views, likes, media_type, comments, link, caption

In [297]:
data = []
for element in elements_to_hover_over[:5]:
    sleep(1)
    data.append(list(get_info(hover_css, element)))

With scrolling

In [277]:
browser.get(insta_url +new_df.brand_ig[0])

In [325]:
ele = browser.find_elements_by_css_selector('div[class="_mck9w _gvoze _tn0ps"]')

In [255]:
len(ele)

12

In [15]:
def scroll_page_once():
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [266]:
browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
ele = browser.find_elements_by_css_selector('div[class="_mck9w _gvoze _tn0ps"]')
len(ele)

42

In [274]:
browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [280]:
ele = browser.find_elements_by_css_selector('div[class="_mck9w _gvoze _tn0ps"]')
len(ele)

12

In [16]:
def scrap_all():
    data = []
    urls = set()
    prev_data_len = -1
    repeat_counter = 0
    while repeat_counter < 3:
        elements_to_hover_over= browser.find_elements_by_css_selector('div[class="_mck9w _gvoze _tn0ps"]')
        for element in elements_to_hover_over[::-1]:
            sleep(1)
            info = list(get_info(hover_css, element))
            if info[-2] in urls:
                repeat_counter += 1
                break
            urls.add(info[-2])
            repeat_counter = 0
            data.append(info)
        scroll_page_once()
    return data

In [356]:
data = scrap_all()

In [357]:
samsung_df = pd.DataFrame(data, columns=["views","likes","media","comments","url","caption"])

In [358]:
samsung_df.shape

(263, 6)

In [361]:
samsung_df.to_csv("samsung_ig.csv",index=False)

In [362]:
samsung_df.head(10)

,views,likes,media,comments,url,caption
0,na,33.8k,image,240,https://www.instagram.com/p/BeIbe5ehQJo/?taken...,May the year ahead run in parallel with your d...
1,na,44.5k,image,389,https://www.instagram.com/p/BeQKCUUBhiX/?taken...,Never lose track of the little moments in life...
2,na,66.5k,image,541,https://www.instagram.com/p/BeSur9mhhqQ/?taken...,Great moments are often framed by the little o...
3,na,57k,image,645,https://www.instagram.com/p/BeVUHj_B_d0/?taken...,"No story too large, no memory too small. #Gala..."
4,na,48.7k,image,"1,031",https://www.instagram.com/p/BeXjt2YhnJS/?taken...,"Your brand new baby is coming on 02.25.2018, a..."
5,na,46.6k,image,739,https://www.instagram.com/p/BeiNayshH89/?taken...,This 02.25.2018. You’re going to be exposed to...
6,na,58.7k,image,"1,324",https://www.instagram.com/p/BenW_EdhAXh/?taken...,From 02.25.18. You won't see your beloved the ...
7,na,38k,image,971,https://www.instagram.com/p/Be2yoNwhX4b/?taken...,You must first walk through darkness to reach ...
8,na,57.2k,image,11.5k,https://www.instagram.com/p/Be5Ws4Ohw5Y/?taken...,"Find beauty, even on the rockiest of roads. #G..."
9,na,53.2k,image,"1,504",https://www.instagram.com/p/Be-g0diBSVq/?taken...,Only when you step out of the grid can you get...


class="_m10kk" <- to get likes from videos after clicking on the views element

In [365]:
#visit url
samsung_df.loc[0].url

'https://www.instagram.com/p/BeIbe5ehQJo/?taken-by=samsungmobile'

In [381]:
#get datetime
browser.find_element_by_tag_name("time").get_attribute("datetime")

'2018-01-19T12:52:08.000Z'

In [382]:
#get likes
browser.find_element_by_css_selector('a[class="_nzn1h"]').find_element_by_tag_name("span").text.replace(",","")

'33893'

In [376]:
#get views of video
browser.find_element_by_css_selector('span[class="_m5zti"]').find_element_by_tag_name("span").text.replace(",","")

'111270'

In [378]:
#click views to review likes
browser.find_element_by_css_selector('span[class="_m5zti"]').click()

In [380]:
#get video likes
browser.find_element_by_css_selector('div[class="_m10kk"]').find_element_by_tag_name("span").text.replace(",","")

'17977'

In [392]:
samsung_df["post_datetime"] = None

In [400]:
for i in range(samsung_df.shape[0]):
    row = samsung_df.loc[i]
    url = row.url
    browser.get(url)
    row.post_datetime = browser.find_element_by_tag_name("time").get_attribute("datetime")
    if row.media == "image":
        row.likes = browser.find_element_by_css_selector('a[class="_nzn1h"]').find_element_by_tag_name("span").text.replace(",","")
    else:
        row.views = browser.find_element_by_css_selector('span[class="_m5zti"]').find_element_by_tag_name("span").text.replace(",","")
        browser.find_element_by_css_selector('span[class="_m5zti"]').click()
        sleep(0.5)
        row.likes = browser.find_element_by_css_selector('div[class="_m10kk"]').find_element_by_tag_name("span").text.replace(",","")

In [402]:
samsung_df.tail(100)

,views,likes,media,comments,url,caption,post_datetime
163,na,53050,image,489,https://www.instagram.com/p/BSj9BMzhtsf/?taken...,The #GalaxyS8 is right at home in one of the m...,2017-04-06T23:07:57.000Z
164,na,24539,image,108,https://www.instagram.com/p/BSmAzB_BfnK/?taken...,Swipe right to see the parallels between a bea...,2017-04-07T18:19:27.000Z
165,na,29484,image,133,https://www.instagram.com/p/BSoGpg_BnuD/?taken...,A camera that captures every angle gives you a...,2017-04-08T13:49:03.000Z
166,87816,15357,video,115,https://www.instagram.com/p/BSwRlKohWYW/?taken...,Train like there’s no finish line. #GearFit2,2017-04-11T17:58:30.000Z
167,107788,18743,video,113,https://www.instagram.com/p/BSwfYxSB1kM/?taken...,Take your adventure to new heights with the #G...,2017-04-11T19:59:09.000Z
168,na,28379,image,200,https://www.instagram.com/p/BOpGdf5jsC2/?taken...,The holidays can be pretty hectic. Remember to...,2016-12-30T13:01:05.000Z
169,na,42600,image,439,https://www.instagram.com/p/BOw0ujoDyT1/?taken...,Here's to shining brightly in #2017. #lights #...,2017-01-02T13:00:03.000Z
170,na,27595,image,253,https://www.instagram.com/p/BO2FfpVDh9w/?taken...,Where will 2017 take you? @LonelyPlanet #ShotF...,2017-01-04T14:02:46.000Z
171,na,24151,image,117,https://www.instagram.com/p/BO7H6-CjfRN/?taken...,The weather outside is delightful. Make a reso...,2017-01-06T13:00:10.000Z
172,na,34903,image,397,https://www.instagram.com/p/BPDLZXVD0EM/?taken...,Rollin' into #2017 like... #GalaxyS7edge #neon...,2017-01-09T16:04:28.000Z


In [403]:
samsung_df.to_csv("samsung_ig_final.csv",index=False)

In [34]:
new_df.n_posts = new_df.n_posts.apply(lambda x: 0 if x=="-" else int(x))

In [408]:
new_df["done"] = None

In [18]:
todo = new_df[new_df.done != 1][new_df.brand_ig != "-"].sort_values(by=["n_posts"]).brand_ig

/home/nikolas/anaconda3/envs/main/lib/python3.5/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [19]:
todo = todo.reset_index(drop=True)

In [49]:
todo = todo[-1:]

In [50]:
todo

28    oneplus
Name: brand_ig, dtype: object

In [51]:
for ig in todo:
#     stage 1
    print(ig)
    browser.get(insta_url +ig)
    data = scrap_all()
    df = pd.DataFrame(data, columns=["views","likes","media","comments","url","caption"])
    df.to_csv(ig + "_ig.csv", index=False)
#     df["post_datetime"] = None
#     #stage 2
#     for i in range(0,df.shape[0]):
#         if i % 10 == 0:
#             print(i)
#         sleep(1)
#         row = df.loc[i]
#         url = row.url
#         browser.get(url)
#         sleep(0.5)
#         row.post_datetime = browser.find_element_by_tag_name("time").get_attribute("datetime")
#         if row.media == "image":
#             try:
#                 row.likes = browser.find_element_by_css_selector('a[class="_nzn1h"]').find_element_by_tag_name("span").text.replace(",","")
#             except:
#                 continue
#         else:
#             try:
#                 row.views = browser.find_element_by_css_selector('span[class="_m5zti"]').find_element_by_tag_name("span").text.replace(",","")
#                 browser.find_element_by_css_selector('span[class="_m5zti"]').click()
#                 sleep(0.5)
#                 row.likes = browser.find_element_by_css_selector('div[class="_m10kk"]').find_element_by_tag_name("span").text.replace(",","")
#             except:
#                 continue
#     df.to_csv(ig + "_ig_final.csv",index=False)
#     new_df.loc[new_df[new_df.brand_ig==ig].index.values[0]]["done"] = 1
#     new_df.to_csv("brand_ig_data.csv",index=False)

oneplus


In [52]:
df

,views,likes,media,comments,url,caption
0,na,17k,image,135,https://www.instagram.com/p/Bg4Lzh0gLIg/?taken...,Paint me a picture with your true colors 🖌#Sho...
1,na,18.1k,image,61,https://www.instagram.com/p/Bg4nLicgLtW/?taken...,The way out. 🏃Excellent #londonist capture by ...
2,na,23.5k,image,96,https://www.instagram.com/p/Bg5eJl5A-PU/?taken...,🌇 Enjoy a sunset in Italy with @matteobtt and ...
3,na,28.3k,image,75,https://www.instagram.com/p/Bg6VKpUA_w_/?taken...,Capture the light with the #OnePlus5T. 🌞😍 #Sho...
4,na,27.3k,image,82,https://www.instagram.com/p/Bg6wnT6HDQr/?taken...,"🇮🇹 When in Rome, do as @pieromod does and take..."
5,na,25.2k,image,300,https://www.instagram.com/p/Bg8C7HJglPc/?taken...,"✨Start your day right with a latte, a notebook..."
6,na,18.4k,image,68,https://www.instagram.com/p/Bg855rCgSpp/?taken...,🏊Cloudy days won't stop us from enjoying a day...
7,na,21.6k,image,94,https://www.instagram.com/p/Bg9VX0UABeA/?taken...,👌🏼Stunning #OnePlus5T captures of Spiti Valley...
8,na,22.2k,image,110,https://www.instagram.com/p/Bg9wxvygf-B/?taken...,🌲Get lost. 🌲#ShotonOnePlus by @roguerambleruk ...
9,na,22.3k,image,343,https://www.instagram.com/p/Bg_esHwAlKl/?taken...,Three cheers for the weekend! 😍 Tell us your p...


In [66]:
df.caption[2]

'🌇 Enjoy a sunset in Italy with @matteobtt and the #OnePlus5 😍. Can you guess the location of this spot? 🤔#ShotonOnePlus\n***\n#passionpassport #beautifuldestinations #travellingthroughtheworld #heavenly_shotz #JustGoShoot #europe #TheImaged #animalsofinstagram #postcardsfromtheworld #tv_pointofview #Rsa_Outdoors #Eclectic_Shotz #CityKillerz  #kings_alltags #bnw_mroldschool'

In [73]:
import extractPhoneModels

In [71]:
getBrands("Google")

set()

In [74]:
import re

In [75]:
rgx = re.compile("[\w+]+")

In [78]:
rgx.findall("Google")

['Google']

In [90]:
def getBrands(text):
    text = text.lower()
    brandsFound = set()
    words = set(rgx.findall(text))
    for brand in brands:
        if brand.lower() in words:
            brandsFound.add(brand)
    return brandsFound


In [80]:
import re, time, json

with open("assets/model-fragments-small.json", "r") as file:
    fragmentLookup = json.load(file)
with open("assets/model-all-tokens-small.json", "r") as file:
    allModelTokens = json.load(file)
with open("assets/model-brands-small.json", "r") as file:
    brands = json.load(file)


In [96]:

MAX_FRAGMENT_SIZE=0 # so that we don't have to search till the end of the string
for frag, index in fragmentLookup.items():
    if frag.count(" ") + 1 > MAX_FRAGMENT_SIZE:
        MAX_FRAGMENT_SIZE = frag.count(" ") + 1


In [93]:
def getModels(text):
    text = text.lower()
    modelsFound = set()
    fragmentsFound = set()
    fragmentsAccepted = {}
    
    words = rgx.findall(text)
    for i in range(0, len(words)):
        if words[i] not in allModelTokens:
            continue
        for j in range(i+1, min(i+1+MAX_FRAGMENT_SIZE, len(words)+1)):
            fragment = " ".join(words[i:j])
            if fragment in fragmentLookup:
                fragmentsFound.add(fragment)
    for fragment in sorted(fragmentsFound, key=len, reverse=True):
        # we iterate through the fragments we've found, to ensure that 'smaller' 
        # model names are not accidentally extracted (e.g. Pixel vs Pixel 2)
        found = False
        for frag, index in fragmentsAccepted.items():
            if fragment in frag and text.find(frag) + frag.find(fragment) == text.find(fragment):
                found = True
                break
        if not found:
            fragmentsAccepted[fragment] = text.find(fragment)
    for frag, index in fragmentsAccepted.items():
        modelsFound.add(fragmentLookup[frag])
    return modelsFound

In [98]:
getBrands(df.caption[0])

set()

In [100]:
rgx.findall(df.caption[0])

['Paint',
 'me',
 'a',
 'picture',
 'with',
 'your',
 'true',
 'colors',
 'ShotonOnePlus',
 'with',
 'the',
 'OnePlus5T',
 'by',
 'alfierecarte',
 'in',
 'madrid',
 'spain',
 'What',
 's',
 'your',
 'favourite',
 'color',
 'whpcolor',
 'spain_insta',
 'igmadrid',
 'beautifuldestinations',
 'europe',
 'jj_mobilephotography',
 'passionpassport',
 'color',
 'vacation',
 'way2ill',
 'wanderlust']